In [1]:
from pathlib import Path

import ast

import re

import operator

from functools import reduce

In [2]:
data_path = Path.home() / 'workstation' / 'dev' / 'Advent-of-Code-2020' / 'data' / 'day18_input.txt'

In [3]:
data_path.exists()

True

In [4]:
with open(data_path, 'r') as reader:
    expr_input = reader.read().strip()

In [5]:
expressions = expr_input.split('\n')

#### Part 1

In [6]:
def evaluate(expression):
    IGNORE_CHAR = [' ']
    intermediate_numbers = []
    intermediate_operators = []
    OPERATORS = {
        '+': lambda y, x: x + y,
        '*': lambda y, x: x * y
    }
    
    index = 0
    while index in range(len(expression)):
        token = expression[index]

        if token in OPERATORS:
            intermediate_operators.append(token)
            index += 1
            
        elif token == '(':
            start_index = index+1
            substring = expression[start_index: ]
            end_index, intermediate_result = evaluate(substring)
            intermediate_numbers.append(intermediate_result)
            index += end_index+2
            
        elif token == ')':
            return index, intermediate_numbers[-1]
            
        elif token not in IGNORE_CHAR:
            intermediate_numbers.append(int(token))
            index += 1
            
        else:
            index += 1
            
        if len(intermediate_numbers) == 2:
            operator = intermediate_operators.pop()
            intermediate_numbers.append(OPERATORS[operator](
                intermediate_numbers.pop(), intermediate_numbers.pop()
            ))
    
    return intermediate_numbers[-1]

In [7]:
sum(evaluate(expr) for expr in expressions)

14006719520523

##### Alternative approach via AST (From Peter Norvig)

In [8]:
example = """((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2"""

In [9]:
# Tokenization of a sort, by inserting commas between operators

re.sub('([+*])', r",'\1',", example)

"((2 ,'+', 4 ,'*', 9) ,'*', (6 ,'+', 9 ,'*', 8 ,'+', 6) ,'+', 6) ,'+', 2 ,'+', 4 ,'*', 2"

In [10]:
def parse_expr(line) -> tuple: 
    "Parse an expression: '2 + 3 * 4' => (2, '+', 3, '*', 4)."
    return ast.literal_eval(re.sub('([+*])', r",'\1',", line))

operators = {'+': operator.add, '*': operator.mul}

def evaluate(expr) -> int:
    "Evaluate an expression under left-to-right rules."
    if isinstance(expr, int):
        return expr
    else:
        a, op, b, *rest = expr
        x = operators[op](evaluate(a), evaluate(b))
        return x if not rest else evaluate((x, *rest))

In [11]:
parse_expr(example)

(((2, '+', 4, '*', 9), '*', (6, '+', 9, '*', 8, '+', 6), '+', 6),
 '+',
 2,
 '+',
 4,
 '*',
 2)

In [12]:
evaluate(parse_expr(example))

13632

#### Part 2 (using Norvig's solution above as I don't think my approach will generalize without a lot of effort)

Basic idea: Use an AST. My approach not likely to work owing to the fact that I didn't tokenize the string. The AST evaluation renders each parenthetical expression as a separate element of a list.

In [13]:
def evaluate_2(expr):
    if isinstance(expr, int):
        return expr
    else:
        while '+' in expr:
            expr = list(expr)
            index = expr.index('+')
            a = expr[index-1]
            b = expr[index+1]
            expr[index-1: index+2] = [evaluate_2(a) + evaluate_2(b)]
        # Need to put evaluate_2 here as multiplication isn't covered in the while loop
        return reduce(lambda x, y: x*y, [evaluate_2(x) for x in expr if x != '*'])

In [14]:
sum(evaluate_2(parse_expr(expr)) for expr in expressions)

545115449981968